### Lorentz System

$\frac{\mathrm{d}x}{\mathrm{d}t} = \sigma (y - x), \\[6pt]$


$\frac{\mathrm{d}y}{\mathrm{d}t} = x (\rho - z) - y, \\[6pt]$


$\frac{\mathrm{d}z}{\mathrm{d}t} = x y - \beta z.$

$σ = 10$, $β = 8/3$ and $ρ = 28$

In [11]:
import autograd.numpy as np
from autograd import grad, elementwise_grad
from matplotlib import pyplot as plt

### Activation Functions

In [12]:
def sigmoid(z) -> float:
    return 1 / (1 + np.exp(-z))


def tanh(z) -> float:
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))


def elu(z, alpha: float) -> float:
    return alpha * (np.exp(z) - 1) if z < 0 else z

### Trial Solution & Analytical Solution

In [114]:
def trial_solution(
    input: np.array, parameters: list, initial_conditions: np.array
) -> np.array:
    print("flag 1")
    print(initial_conditions)
    print(input)
    print(forward_propogation(input, parameters))
    print(input * forward_propogation(input, parameters))
    return initial_conditions + input * forward_propogation(input, parameters)


def right_hand_side(
    input: np.array, trial_sol: np.array, constants: np.array
) -> np.array:
    return np.array(
        [
            constants[0] * (trial_sol[1] - trial_sol[0]),
            trial_sol[0] * (constants[2] - trial_sol[2]) - trial_sol[1],
            trial_sol[0] * trial_sol[1] - constants[1] * trial_sol[2],
        ]
    )


def lorenz(xyz, *, s=10, r=28, b=2.667):
    """
    Parameters
    ----------
    xyz : array-like, shape (3,)
       Point of interest in three-dimensional space.
    s, r, b : float
       Parameters defining the Lorenz attractor.

    Returns
    -------
    xyz_dot : array, shape (3,)
       Values of the Lorenz attractor's partial derivatives at *xyz*.
    """
    x, y, z = xyz
    x_dot = s * (y - x)
    y_dot = r * x - y - x * z
    z_dot = x * y - b * z
    return np.array([x_dot, y_dot, z_dot])

### Loss Function

In [115]:
def MSE_loss_function(
    input, parameters: list, constants: np.array, initial_conditions: np.array
):
    print("flag 0")
    trial_sol = trial_solution(input, parameters, initial_conditions)
    print("flag 2")
    prediction = right_hand_side(input, trial_sol, constants)
    neural_network_gradient = elementwise_grad(trial_solution, 0)(
        input, parameters, initial_conditions
    )


    error_squared = (neural_network_gradient - prediction) ** 2

    loss_sum = np.sum(error_squared)


    return loss_sum / np.shape(error_squared)[0]

### Neural Network

In [116]:
def init_parameters(input_size: int, hidden_sizes: np.array, output_size: int):
    np.random.seed(0)

    # hidden weights and biases
    hidden_weights = [np.random.randn(hidden_sizes[0], input_size)]
    for i in range(1, hidden_sizes.shape[0]):
        hidden_weights.append(np.random.randn(hidden_sizes[i], hidden_sizes[i - 1]))

    hidden_biases = [np.zeros((h, 1)) for h in hidden_sizes]

    # output weights and biases
    output_weights = np.random.randn(output_size, hidden_sizes[-1])
    output_bias = np.zeros((output_size, 1))

    parameters = [hidden_weights, hidden_biases, output_weights, output_bias]
    return parameters

In [124]:
def forward_propogation(input: np.array, parameters: list):
    hidden_weights, hidden_biases, output_weights, output_bias = parameters
    num_hidden_layers = len(hidden_weights)

    # hidden layers
    z = np.matmul(hidden_weights[0], input) + hidden_biases[0]
    a = sigmoid(z)
    for i in range(1, num_hidden_layers):
        z = np.matmul(hidden_weights[i], a) + hidden_biases[i]
        a = sigmoid(z)

    # output layer
    z = np.matmul(output_weights, a) + output_bias
    a = sigmoid(z)
    return a

In [125]:
parameters = init_parameters(1, np.array([2, 3, 4]), 3)
len(parameters)

4

In [126]:
def back_propogation(
    input: np.array,
    parameters: list,
    constants: np.array,
    initial_conditions: np.array,
    num_iter: int,
    learn_rate: float,
):
    loss_grad_function = grad(MSE_loss_function, 1)

    for i in range(num_iter):
        loss_grad = loss_grad_function(input, parameters, constants, initial_conditions)
        return loss_grad.shape
        # hidden layers
        for j in range(len(parameters[0])):
            parameters[j] = parameters[j] - learn_rate * loss_grad[j]

    return parameters

In [127]:
back_propogation(
    np.array([1]), parameters, np.array([1, 1, 1]), np.array([0, 0, 0]), 1, 1
)

flag 0
flag 1
[0 0 0]
[1]
Autograd ArrayBox with value [[0.39456145 0.39020071]
 [0.49644285 0.49879528]
 [0.9190612  0.91674261]]
Autograd ArrayBox with value [[0.39456145 0.39020071]
 [0.49644285 0.49879528]
 [0.9190612  0.91674261]]
Autograd ArrayBox with value [[0.39456145 0.39020071]
 [0.49644285 0.49879528]
 [0.9190612  0.91674261]]
Autograd ArrayBox with value [[0.39456145 0.39020071]
 [0.49644285 0.49879528]
 [0.9190612  0.91674261]]
Autograd ArrayBox with value [[0.39456145 0.39020071]
 [0.49644285 0.49879528]
 [0.9190612  0.91674261]]


ValueError: operands could not be broadcast together with shapes (3,) (3,2) 